# DABN13 - Assignment 3
## Preamble: Forecasting market returns
We continue with the problem of forecasting market return that we illustrated in Lecture 5. In order to train the implementation of PCR/PLS, we will replicate our previous results with a slight twist. More specifically, we will choose tuning parameters via cross-validation. Additionally, the training sets used for model evaluation are defined differently. While the example in Lecture 5 defined an expanding window of training sets with fixed starting period, we are going to use a rolling window that moved both the start and end date of the training set.

The data for this lab is provided by two csv files. 
The file *sorted_portfolios100.csv* contains the monthly returns of 100 equally weighted portfolios sorted by size and the book-to-market ratio. The data is taken from Kenneth French's data library and missing values have been inserted. The time period covered is January 1960 until December 2009
The file *twelve_month_returns.csv* contains 12-month returns on a value-weighted market portfolio. This series takes moving 12-month sums of returns of the U.S. market factor, as provided on Kenneth French's data library. The entry in row $t$ of the dataset corresponds to the market returns over the months $t+1$ until $t+12$. Accordingly, the first observed value in our sample is the 12-month return over the period February 1960 - January 1961. The last observation covers the period January-December 2010.

To begin with the lab, we import both your outcome as well as the 100 predictors into Python using the code below. You might be required to modify the file path.

In [5]:
import numpy as np
import pandas as pd
from os import chdir

#chdir('?') Adjust directory here
portfolios  = pd.read_csv("sorted_portfolios100.csv").iloc[:,1:]
mkt_ret_12m = pd.read_csv("twelve_month_returns.csv").iloc[:,1:] 

mkt_ret_12m_train = mkt_ret_12m.iloc[:540,:]
portfolios_train  = portfolios.iloc[:540,:]

## Part 1: Setting up a PCR pipeline in `sklearn`

Principal component regression is not available in scikit-learn as a distinctive learning algorithm. This is no problem because it is the combination of three simple components:

1. Standardizing inputs,
2. obtaining $k$ principal components from the inputs,
3. running a linear regression of the output on the $k$ principal components (and a constant).

All three operations above are implemented in sklearn. In order to put them together, we will use the very convenient `Pipeline()` functionality.

Pipelines allow us to specify a custom sequence of actions. An existing pipeline can be fit, similar to the specification of a regression model. If we do that, all steps of the pipeline are executed in their specified order.

### Task 1a)

In this task, we will create and fit a pipeline for PCR. First, we need to specify the three actions mentioned above individually. This can be done by using the `StandardScalar()`, `PCA()` and `LinearRegression()` functions from several `sklearn` modules. I have already written code below to import these functions. Now, please create three objects `stdz_1a`, `PCA_1a` and `lm_1a` which specify

1. input standardization,
2. PCA with two principal components,
3. a linear regression model,

respectively.

Next, we use the `Pipeline()`-function to define the full sequence of actions that result in PCR. I already prepared the basic structure of the required line of code. All you need is to specify the three steps of our pipeline and to give them suitable names. Please do that and save the resulting object as `pcr_pipe_1a`.

Lastly, we can apply the `fit()`-method to `pcr_pipe_1a` to execute the entire pipeline. Do this using `mkt_ret_12m_train`and `portfolios_train` as your output and inputs, respectively. Save the learned model as `pcr_fit_1a`. 

In [1]:
# CODE_CHUNK code_chunk_01
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

stdz_1a = ??
PCA_1a  = ??
lm_1a   = ??

pcr_pipe_1a = ??
pcr_fit_1a  = ??
 

SyntaxError: invalid syntax (1352952626.py, line 7)

### Task 1b)

We will now get predictions from our learned principal components regression. However, we'll do that time series style. 

When we work with time series, our training data consists of historical series from some starting point in the past until today. Our goal is outcome prediction in the immediate future. That is, our test data may simply be a single data point - at least for the model that we trained at one specific point in time. 

In Task 1a) we used the earliest 540 data points, assuming that this was all our data. Now, we want to get a forecast of data point 541, assuming that this is the immediate future. The more distant future is not of immediate interest for us and we will predict it step-by-step as time passes and more training data appears. 

Now apply the `predict()` method on `pcr_fit_1a` in order to get a prediction from your model for data point 541 in `portfolios_train` and save it as `pcr_pred_1b`.

*Note:* Being able to apply `predict()` to a pipeline is very convenient because we do not have to worry about manual standardization and manual construction of the principal component scores. We simply provide the raw test data point and `sklearn` automatically conducts all necessary transformations. 

In [ ]:
# CODE_CHUNK code_chunk_02
??

## Part 2: Model tuning with `sklearn`

Scikit-learn contains very flexible routines for model tuning. The tuning parameter of PCR is the number of principal components to use. We will use the PCR pipeline created in the previous part to choose it via cross-validation.

### Task 2a)

First, we need to define a grid of potential tuning parameter values. This has to be a *dictionary* object in Python which contain arrays with the candidate values for our tuning parameters. The names of the arrays must be identical to the name of the tuning parameter in the corresponding estimator function.

Given that we tune the parameters of an entire pipeline, the names of arrays must also point at which part of the pipeline the tuning parameter can be found. To give an example, assume that we want to tune an imaginary parameter `myparam` in a function that included in my pipeline under the name `myfun`. Then, the correspoding array in the dictionary object must have the name `myfun__myparam`. 

Now create a dictionary object `tune_grid_2a` that contains an array with the integers $1,2,3,4,5$ for the number of principal components to obtain in the PCA-step of your pipeline `pcr_pipe_1a`. 

In [ ]:
# CODE_CHUNK code_chunk_03
??

### Task 2b)
Next, we must specify how exactly our cross-validation procedure should split the data into training and holdout folds. This is done by specifying a splitter class. 

Since our ultimate goal is to predict market returns in the immediate future, this goal should be mimicked by our cross-validation splits. Hence, we are using the `TimeSeriesSplit()`-function which defines many splits into historical training data and future test data.

Now use `TimeSeriesSplit()` to define an object `cv_splits_2b` which contains a specification of the desired splits. To be specific, we want the following:

- Training folds should always consist of 90 data points.
- The test fold following the training folds should only contain 1 data point.
- We want a total of 450 splits, so that the training fold window moves through the entire length of `mkt_ret_12m_train` and `portfolios_train`.
- No gap between training and test folds.

Please check the  `TimeSeriesSplit()`-[documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html#sklearn.model_selection.TimeSeriesSplit) to figure out how to set the function inputs.

In addition to that, apply the `split()`-method to `cv_splits_2b` *and* turn the resulting output into a list object that you save as `cv_splits_list_2b`. Use `portfolios` as the `X`-argument of `split()`.

Lastly, look at the items inside `cv_splits_list_2b`. They contain the indexes of data points that are part of training and hold-out folds for every split. Use the string variable `split_characterization_2b` to characterize the pattern that you see over your 450 splits.

In [ ]:
# CODE_CHUNK code_chunk_04
from sklearn.model_selection import TimeSeriesSplit

# 1.
cv_splits_2b     = TimeSeriesSplit(??)

# 2.
cv_splits_list_2b = ??

# 3. 
split_characterization_2b = "??"

### Task 2c)
Tasks 1a, 2a, and 2b provide us with all the components that we need to specify and to execute a cross-valdation based tuning parameter selection.

First, use the `GridSearchCV()` function in the `model_selection` module of Scikit-learn to specify the setup for model tuning. To be more specific, we want the following:

- We want to tune the model pipeline from Task 1a (don't worry about the number of principal components specified there. It will be overwritten),
- we use the grid of tuning parameter values from Task 2a,
- cross-validation is conducted based on the splits defined in `cv_splits_2b`,
- performance should be evaluated using the (negative) root mean square error.

Look up the `GridSearchCV()`-[documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to figure out how to set the function arguments correctly and save the resulting setup for model tuning as `pcr_tune_2c`.

Second, run the tuning procedure by applying the `fit()`-method to `pcr_tune_2c`. Use `portfolios_train` and `mkt_ret_12m_train` as inputs and output, respectively.


In [ ]:
# CODE_CHUNK code_chunk_05
from sklearn.model_selection import GridSearchCV


# 1.
pcr_tune_2c = GridSearchCV(??)

# 2.
pcr_tune_2c.fit(??)


### Task 2d)

`pcr_tune_2c` contains both the optimal tuning parameter value as well as the learned model with this value.  Where inside `pcr_tune_2c` can you find these two objects? Write your specific(!) answer into the string variable `cv_results_2d`.

Additionally, extract the `cv_results_` dictionary from `pcr_tune_2c`, which emerges once we applied the `fit()`-method, into a new object called `pcr_tune_details_2d`. Where inside `pcr_tune_details_2d` can we find the estimated cross-validation estimate of test error for the different tuning parameter candidates? Write your answer into the string variable "test_err_estimates_2d".

In [ ]:
# CODE_CHUNK code_chunk_06
# 1.
cv_results_2d         = "??"

# 2.
pcr_tune_details_2d   = ??
test_err_estimates_2d = "??"

### Task 2e) 
As an alternative to PCR, we consider PLS for return forecasting. PLS is implemented in `sklearn` in the form of the `PLSRegression()` function. Now do the following:

1. Build a pipeline that first standardizes the inputs and then conducts PLS. In the PLS-part, please do not specify the number of PLS directions, but disable variable scaling. Save the pipeline as `pls_pipe_2e`.
2. Create a tuning parameter grid that contains $1,2,3,4,5$ PLS directions as candidate values. Save it as `tune_grid_2e`.
3. Use `GridSearchCV()` to choose the optiimal number of PLS directions. Use the same setup as in Task 2c. Save the resulting object as `pls_tune_2e` and don't forget to fit this object.

In [ ]:
# CODE_CHUNK code_chunk_07
from sklearn.cross_decomposition import PLSRegression

# 1.
pls_pipe_2e  = ??

# 2. 
tune_grid_2e = ??

pls_tune_2e  = ??
??


### Task 2f)
Which tuning parameter choices do we get for PCR and PLS? Write your answer into the string variable `tune_best_2g`. Additionally, save the learned models with best tuning parameters as `pcr_fit_tuned_2f` and `pls_fit_tuned_2f`, respectively.

In [ ]:
# CODE_CHUNK code_chunk_08
tune_best_2g = "??"
pcr_fit_tuned_2f = ??
pls_fit_tuned_2f = ??

## Part 3: Comparing two candidate algorithms

Part 2 helped us to find the optimal tuning parameter values for both PCR and PLS. However, which of these two (tuned) algorithms is better for forecasting returns? We will find out in this rather difficult part.

In order to choose between PCR and PLS, we check how well either of the two procedures predicts the outcomes of data points $541, 542, \ldots, 600$. We still do this in time-series fashion. That means we use data up to data point 540 to predict data point 541 and move start and end of training and hold-out data to the end of the sample one data point at a time. 
The data used to train our model is still the 90 data points prior to the test data point.

### Task 3a)

Create a vector `mkt_ret_12m_test` and a matrix `portfolios_test` that we can use for performance evaluation on the last 60 data points of `mkt_ret_12m` and `portfolios`. That is, it must go back long enough in time to include the training data for an output prediction of data point 541.

In [ ]:
mkt_ret_12m_test = mkt_ret_12m.iloc[450:,:]
portfolios_test  = portfolios.iloc[450:,:]

### Task 3b)

Use `cross_validate()` in sklearn to evaluate the performance of PCR and PLS with the tuning parameter values that you arrived at in Task 2f. `cross_validate()` conducts cross-validation with specified data and a desired scheme for splitting it without any grid search.

First, use `TimeSeriesSplit()`to specify splits into training and holdout folds that contain 90 and 1 data points, respectively. Save this specification as `cv_splits_3a`. The training window should move through the data points in the test data created in Task 3a one data point at a time. The holdout fold of the first split should be the 91st observation. The holdout fold of the last split should be the very last observation in the test data.

Second, use `cross_validate()` to estimate the test error of your tuned PCR and PLS algorithms from Task 2f and use root mean squared error to measure accuracy. Save the resulting objects as `pcr_eval_3b` and `pls_eval_3b`, respectively. Check the `cross_val_score()`-[documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) to get the function inputs right.

Third, check which object inside `pcr_eval_3b` or `pcr_eval_3b` contains the root mean squared errors of all holdout folds. Write your answer into the string variable `where_is_rmse_3b`. Since we only have one data point in every holdout fold, the RMSE is identical to the absolute value of our prediction error on the test fold.



In [ ]:
# CODE_CHUNK code_chunk_09
from sklearn.model_selection import cross_validate 

# 1.
cv_splits_3b = ??

# 2.
pcr_eval_3b = cross_validate(??)

pls_eval_3b = ??

where_is_rmse_3b = "??"


### Task 3c)

The RMSE provided by `cross_validate()` is hard to interpret if we want to know whether any of the two machine learning algorithms is practically useful at all. A better alternative is out-of-sample R-squared. We define it mathematically as 
$$
  R^2 = 1 - \frac{ \sum_{t=t_0}^{T}(y_t - \hat{y}_t)^2}{ \sum_{t=t_0}^{T}(y_t -\bar{y}_{t,-90})^2}.
$$
Here, 

- $y_t$ is the observed output at time $t$, 
- $t_0$ is the time period of the earliest test observation (541 in our case),
- $\hat{y}_t$ is the model prediction of $y_t$ at time $t$,
- $\bar{y}_{t,-90}=90^{-1}\sum_{\ell}^{90} y_{t-\ell}$ is the average outcome of the 90 output values before $t$

To construct this R-squared, do the following

1. Construct a vector `ymeans_3c` whose element $t$ contains the average of `mkt_ret_12m` from time point $450+t$ to $539+t$. You could apply the `rolling()` method to `mkt_ret_12m_test` or write a for-loop to achieve that.
2. Extract the sum of squared prediction error of PLS and PCR over all test folds from `pcr_eval_3b` and `pls_eval_3b`. Save them as numbers `pcr_avg_errsq_3c` and `pls_avg_errsq_3c`, respectively.
4. Construct the summed squared prediction error of the historical 90-day average output but using `mkt_ret_12m_test` and `ymeans_3c`. Save the resulting number as `histavg_avg_errsq_3c`.
5. Construct out-of-sample R2 for PCR and PLS using the objects created in the previous two steps. Save them as `pcr_R2_oos_3c` and `pls_R2_oos_3c`, respectively.

Does PCR outperform PLS or is it the other way around? Does the out-of-sample R-squared suggest that any of the two methods is useful for predicting returns? Motivate your conclusion and express it in the string variable `conclusion_3c`. 

In [ ]:
# CODE_CHUNK code_chunk_10
# 1.
ymeans_3c            = ??

# 2.
pcr_sum_errsq_3c     = ??
pls_sum_errsq_3c     = ??

# 3.
histavg_sum_errsq_3c = ??

# 4.
pcr_R2_oos_3c        = ??
pls_R2_oos_3c        = ??


conclusion_3c = "??"


### Part 4: PCR manually

In this advanced part, we will repeat Part 1 without using the `pca()`-transformation in scikit-learn. More specifically, we will replace it with matrix operations in Numpy.

### Task 4a)

By convention, principal components are obtained from standardized data. In order to do that, we use the `StandardScaler()` function from in scikit-learn that we already made part of the PCR pipeline. 
Statistical standardization using `StandardScaler()` needs to be first specified and then fit on data. Then it can be used to transform any desired dataset. We will do this now:

1. Use `StandardScaler()` to specify and to fit an object that contains sample means and variances of each variable in `portfolios_train`. Save this object as `stdz_X_4a`.
2. Apply the `transform()`-method to `stdz_X_4a` to standardize `portfolios_train`. Save the resulting data frame as a two-dimensional *Numpy array* `X_train_stdz_4a`. 
3. An alternative method for variable scaling would be normalization. Search among the functions [in the preprocessing module](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) of `sklearn` for an equivalent to `StandardScaler()`. Then, conduct the previous two steps using this function. However, when you save objects, use the name `norm_X_4a` instead of `stdz_X_4a` and `X_train_norm_4a` instead of `X_train_stdz_4a`. 

In [6]:
# Ta bort sen
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

In [12]:
# CODE_CHUNK code_chunk_11
# 1.
stdz_X_4a       = StandardScaler().fit(portfolios_train)

# 2.
X_train_stdz_4a = stdz_X_4a.transform(portfolios_train)


# 3.
from sklearn.preprocessing import Normalizer
norm_X_4a       = Normalizer().fit(portfolios_train)
X_train_norm_4a = norm_X_4a.transform(portfolios_train)

### Task 4b)

We can now use the eigendecomposition (a.k.a spectral decomposition) to obtain principal components (PCs). The `eig()`-function in the linear algebra module of NumPy  allows us to obtain a list object containing the eigenvectors and eigenvalues of any square matrix that we feed into `eig()`. Now, we want to construct the scores of the first two principal components. In order to do that, review the slides of Lecture 5 and do the following:

1. Use `eig()` to get the loadings of the first two principal components of `X_train_stdz_4a`. Save them as `PCloadings_4b`.
2. Use `PCloadings_4b` to construct the scores of the first two PCs of `X_train_stdz_4a` and save them as `PCscores_4b`.

In [18]:
# CODE_CHUNK code_chunk_12

# 1.
from numpy.linalg import eig
eigdecomp_4b  = eig(X_train_stdz_4a.T @ X_train_stdz_4a)
PCloadings_4b = eigdecomp_4b.eigenvectors[:,:2]

# 2.
PCscores_4b = X_train_stdz_4a @ PCloadings_4b

Proceed with the second step of PCR: 

1. Create an input matrix `Z_4c` containing `PCscores_4b` and a constant. 
2. Obtain the learned coefficients for a linear regression model with inputs `Z_4c`, output `mkt_ret_12m_train` and squared error loss. Do this manually using only the `solve()` and `transpose()` commands in NumPy as well as matrix operations. Save the result as `pcr_coefs_4c`.

In [20]:
# CODE_CHUNK code_chunk_13
# 1.
Z_4c         = np.hstack([np.ones((PCscores_4b.shape[0], 1)), PCscores_4b])

# 2.
pcr_coefs_4c = np.linalg.solve(Z_4c.T @ Z_4c, Z_4c.T @ mkt_ret_12m_train)

### Task 4c)

Now that we have learned our PCR model, we can get an output prediction for data point 541.

1. Transform the 100 portfolio returns in the 451-st data point of `portfolios` using your fitted standardization object from Task 4a. In order to prevent Python from converting this single row of `portfolios` into a Pandas series, you might add *additional* square brackets around the row that you index. Save the resulting $1\times 100$ NumPy array of standardized inputs as `X_test_4d`.
2. Use `PCloadings_4b` to get the PC scores corresponding to the 100 values in `X_test_4d`. Save them, together with a constant, in a $1\times 3$ NumPy array `Z_test_4d`.
3. Get a prediction from the model learned in Task 4c for the input combination `Z_test_4d`. Save this prediction as `pcr_pred_4d`.

In [21]:
# CODE_CHUNK code_chunk_14
# 1.
X_test_4d = stdz_X_4a.transform([portfolios.iloc[450, :]])

# 2. 
Z_test_4d = np.hstack([np.ones((1, 1)), X_test_4d @ PCloadings_4b])

# 3. 
pcr_pred_4d = Z_test_4d @ pcr_coefs_4c

c:\Users\Sara\anaconda3\envs\STAN51\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
